Tenemos los CSV, vamos a lanzar consultas en SQL, por lo que crearemos una Base de Datos, pasaremos los CSVs a 
tablas dentro de la Base de Datos y así podremos lanzar consultas

In [14]:
# Importar las bibliotecas necesarias
import pandas as pd  # Para manejar archivos CSV y bases de datos
import sqlite3  # Para interactuar con SQLite
import os  # Para comprobar la existencia de archivos

## Comprobar tipo de datos y formato

In [15]:
import pandas as pd

# Lista de archivos CSV en tu proyecto
csv_files = [
    "customers csv.csv",
    "dim_geography.csv",
    "dim_product.csv",
    "dim_territory.csv",
    "fact_sales.csv"
]

# Analizar cada archivo
for file in csv_files:
    print(f"\n🔹 Análisis del archivo: {file}\n")

    # Detectar si es un CSV con delimitador `;`
    with open(file, 'r') as f:
        first_line = f.readline()
        delimiter = ';' if ';' in first_line else ','

    # Cargar CSV con el delimitador correcto
    df = pd.read_csv(file, delimiter=delimiter)

    # Crear DataFrame con nombre de columna, ejemplo de dato y tipo de dato
    analysis = pd.DataFrame({
        "Columna": df.columns,
        "Ejemplo": df.iloc[0].astype(str),  # Convertimos la primera fila a string
        "Tipo de Dato": df.dtypes.values
    })

    # Mostrar tabla en formato más limpio
    from tabulate import tabulate
    print(tabulate(analysis, headers="keys", tablefmt="pretty"))
    print("-" * 50)




🔹 Análisis del archivo: customers csv.csv

+-------------------+-------------------+------------+--------------+
|                   |      Columna      |  Ejemplo   | Tipo de Dato |
+-------------------+-------------------+------------+--------------+
|    customer_id    |    customer_id    |   11000    |    int64     |
|   geography_id    |   geography_id    |     26     |    int64     |
|     birthdate     |     birthdate     | 08/04/1986 |    object    |
|   maritalstatus   |   maritalstatus   |     M      |    object    |
|      gender       |      gender       |     M      |    object    |
| datefirstpurchase | datefirstpurchase | 22/07/2016 |    object    |
+-------------------+-------------------+------------+--------------+
--------------------------------------------------

🔹 Análisis del archivo: dim_geography.csv

+-----------------------------+-----------------------------+-----------------+--------------+
|                             |           Columna           |     

# NUEVAS COLUMNAS A EUROS Y NUEVOS CSV VERSIONADOS

In [16]:
import pandas as pd

# Lista de archivos CSV con los que trabajaremos
csv_files = [
    "dim_product.csv",
    "fact_sales.csv"
]

# Tasa de conversión de Rupiah a Euro
rupiah_to_euro = 0.000055

# Analizar cada archivo y agregar las columnas convertidas
for file in csv_files:
    print(f"\n🔹 Procesando archivo: {file}\n")

    # Detectar si es un CSV con delimitador `;` o `,`
    with open(file, 'r') as f:
        first_line = f.readline()
        delimiter = ';' if ';' in first_line else ','

    # Cargar CSV con el delimitador correcto
    df = pd.read_csv(file, delimiter=delimiter)

    # Comprobamos que los valores sean numéricos antes de aplicar la conversión
    if file == "dim_product.csv":
        # Convertir las columnas cost y normal_price a euros si existen
        if 'cost' in df.columns:
            df['cost_euro'] = (df['cost'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'normal_price' in df.columns:
            df['normal_price_euro'] = (df['normal_price'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)

    elif file == "fact_sales.csv":
        # Convertir las columnas unitprice_rupiah, totalprice_rupiah, totalcost_rupiah, shippingprice_rupiah a euros
        if 'unitprice_rupiah' in df.columns:
            df['unitprice_euro'] = (df['unitprice_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'totalprice_rupiah' in df.columns:
            df['totalprice_euro'] = (df['totalprice_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'totalcost_rupiah' in df.columns:
            df['totalcost_euro'] = (df['totalcost_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'shippingprice_rupiah' in df.columns:
            df['shippingprice_euro'] = (df['shippingprice_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)

    # Guardar el archivo con las columnas convertidas a euros
    df.to_csv(f"converted_{file}", index=False)

    # Mostrar las primeras filas del archivo procesado
    print(f"Primeras filas de {file}:")
    print(df.head(10))




🔹 Procesando archivo: dim_product.csv

Primeras filas de dim_product.csv:
   product_id               product_name          model_name  color  \
0         210  HL Road Frame - Black, 58       HL Road Frame  Black   
1         211    HL Road Frame - Red, 58       HL Road Frame    Red   
2         480        Patch Kit/8 Patches           Patch kit    NaN   
3         529             Road Tire Tube      Road Tire Tube    NaN   
4         477      Water Bottle - 30 oz.        Water Bottle    NaN   
5         528         Mountain Tire Tube  Mountain Tire Tube    NaN   
6         530          Touring Tire Tube   Touring Tire Tube    NaN   
7         484      Bike Wash - Dissolver           Bike Wash    NaN   
8         224               AWC Logo Cap         Cycling Cap  Multi   
9         223               AWC Logo Cap         Cycling Cap  Multi   

  size_range     cost  normal_price       sub_category     category  \
0   54-58 CM  11000.0       11000.0        Road Frames   Components   
1

# 1. Conectar o crear la base de datos SQLite

In [17]:

# Si el archivo 'sales_data.db' no existe, SQLite lo creará automáticamente
conn = sqlite3.connect("sales_data.db")
print("Conexión establecida con la base de datos.")

Conexión establecida con la base de datos.


# 2. Definir los archivos CSV y los nombres de las tablas correspondientes y cargarlos a tablas

In [18]:

import os
import pandas as pd
import sqlite3

# Establecer la conexión con la base de datos SQLite
conn = sqlite3.connect('sales_data.db')

# Crear un cursor
cursor = conn.cursor()

# Creamos un diccionario donde la clave es el nombre de la tabla en la base de datos
# y el valor es el nombre del archivo CSV
csv_files = {
    "customers": "customers csv.csv",
    "dim_customer": "dim_customer.csv",
    "dim_geography": "dim_geography.csv",
    "dim_product": "converted_dim_product.csv",
    "territory": "dim_territory.csv",   # Corregido
    "fact_sales": "converted_fact_sales.csv"   # Corregido
}

# Cargar cada archivo CSV en su tabla correspondiente en SQLite
for table_name, file_name in csv_files.items():
    if os.path.exists(file_name):  # Verificamos que el archivo exista
        df = pd.read_csv(file_name)  # Leer el archivo CSV en un DataFrame
        df.to_sql(table_name, conn, if_exists="replace", index=False)  # Guardar en SQLite
        print(f"Tabla '{table_name}' cargada correctamente con {len(df)} registros.")
    else:
        print(f"El archivo '{file_name}' no se encontró. Verifica su nombre y ubicación.")

# Cerrar la conexión
conn.close()


Tabla 'customers' cargada correctamente con 1360 registros.
Tabla 'dim_customer' cargada correctamente con 1360 registros.
Tabla 'dim_geography' cargada correctamente con 654 registros.
Tabla 'dim_product' cargada correctamente con 397 registros.
Tabla 'territory' cargada correctamente con 11 registros.
Tabla 'fact_sales' cargada correctamente con 5954 registros.


# Exportamos en la terminal sqlite3 sales_data.db .dump > export.sql

Para exportar la base de datos completa a un archivo SQL (incluyendo la estructura de las tablas y los datos) desde la terminal de SQLite, sigue estos pasos:

# Cambiar algunos términos de SQLite para que sea compatible con MySQL
> • Reemplaza todas las ocurrencias de AUTOINCREMENT por AUTO_INCREMENT (MySQL usa AUTO_INCREMENT). <br>
> • Si ves PRAGMA o BEGIN TRANSACTION, elimínalos (MySQL no los usa). <br>
> • Asegúrate de que las FOREIGN KEY tengan ON DELETE CASCADE si quieres que se comporten igual en MySQL. <br>